In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, when, lit, split, concat, regexp_extract



In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("InspectRawData").getOrCreate()


In [ ]:
# Azure Data Lake paths (replace placeholders with actual values)
storage_account_name = "datalakestoragetask"  # Replace with your storage account name
raw_container = "raw"
processed_container = "processed"
storage_key = ""  # Replace with your key or credential method

# Configure Spark to access Azure Data Lake
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_key)

# Define paths for each dataset
paths = {
    "customer_raw": f"abfss://{raw_container}@{storage_account_name}.dfs.core.windows.net/customer",
    "customer_processed": f"abfss://{processed_container}@{storage_account_name}.dfs.core.windows.net/customer/",
}


In [ ]:
# Function to inspect a dataset
def inspect_dataset(name, path, format_type, options={}):
    print(f"\n=== Inspecting {name} Dataset ===")
    try:
        # Load dataset based on format
        df = spark.read.format(format_type).options(**options).load(path)
        
        # Show schema and a sample of the data
        df.printSchema()
        df.limit(30).show(truncate=False)
        
        # Return DataFrame for further analysis if needed
        return df
    except Exception as e:
        print(f"Error reading {name} data: {e}")
        return None

# Inspect datasets one by one
print("\n--- Starting Inspection ---\n")




--- Starting Inspection ---



In [ ]:
# Customer data (CSV format)
customer_df = inspect_dataset(
    name="Customer",
    path=paths["customer_raw"],
    format_type="csv",
    options={"header": True, "delimiter": ";"}  # Added delimiter for semicolon-separated values
)
#customer_df = spark.read.format("csv").options(header=True, delimiter=";").load(paths["customer_raw"])



=== Inspecting Customer Dataset ===
root
 |-- customerID: string (nullable = true)
 |-- User: string (nullable = true)
 |-- SupplierID: string (nullable = true)
 |-- Customername1: string (nullable = true)
 |-- Customername2: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Email1: string (nullable = true)
 |-- Email2: string (nullable = true)
 |-- AdditionalInformation: string (nullable = true)

+----------+----+----------+-------------+-------------+------------------+--------+----------+-------+------+------------------------+---------------------+
|customerID|User|SupplierID|Customername1|Customername2|Street            |Postcode|City      |Country|Email1|Email2                  |AdditionalInformation|
+----------+----+----------+-------------+-------------+------------------+--------+----------+-------+------+------------------------+--------------

In [ ]:
print("=== Inspecting customer Dataset ===")
customer_df.printSchema()
display(customer_df.limit(10).toPandas())  # Display as table-like format

=== Inspecting customer Dataset ===
root
 |-- customerID: string (nullable = true)
 |-- User: string (nullable = true)
 |-- SupplierID: string (nullable = true)
 |-- Customername1: string (nullable = true)
 |-- Customername2: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Email1: string (nullable = true)
 |-- Email2: string (nullable = true)
 |-- AdditionalInformation: string (nullable = true)



customerID,User,SupplierID,Customername1,Customername2,Street,Postcode,City,Country,Email1,Email2,AdditionalInformation
null,null,00001,Tequip,null,Gropiusplatz 10,70563,Stuttgart,DE,null,test.user2@nexmart.com,null
null,null,00002,Tequip,null,Hauptstraße 45,40210,Düsseldorf,DE,null,maria.muster@firma.com,null
null,null,00003,Tequip,null,Karlstraße 12,80331,München,DE,null,franz.meier@firma.com,null
null,null,00004,Tequip,null,Friedrichstraße 22,10969,Berlin,DE,null,lisa.schmidt@firma.com,null
null,null,00005,Tequip,null,Rathausplatz 3,10178,Berlin,DE,null,paul.bauer@firma.com,null
null,null,00006,Tequip,null,Schlossallee 5,01067,Dresden,DE,null,anna.klein@firma.com,null
null,null,00007,Tequip,null,Marktplatz 10,28195,Bremen,DE,null,julia.weber@firma.com,null
null,null,00008,Tequip,null,Bahnhofstraße 8,70173,Stuttgart,DE,null,karl.schulz@firma.com,null
null,null,00009,Tequip,null,Kirchweg 16,60311,Frankfurt,DE,null,martin.huber@firma.com,null
null,null,00010,Tequip,null,Am See 2,78462,Konstanz,DE,null,sophie.fischer@firma.com,null


In [ ]:
#Customer Dataset
#Clean the customer data, handling missing values.

# Handle missing values and create FullAddress column
customer_transformed = customer_df \
    .withColumn(
        "FullAddress", 
        concat(
            when(col("Street").isNotNull(), col("Street")).otherwise(lit("Unknown")), lit(", "),
            when(col("City").isNotNull(), col("City")).otherwise(lit("Unknown")), lit(", "),
            when(col("Country").isNotNull(), col("Country")).otherwise(lit("Unknown"))
        )
    ) \
    .fillna({
        "Email1": "Unknown",  # Fill missing Email1 with "Unknown"
        "Email2": "Unknown"   # Fill missing Email2 with "Unknown"
    }) \
    .filter(col("customerID").isNotNull())  # Ensure critical field `customerID` is not NULL



##4. Write Transformed Data to Processed Folder
Save the transformed datasets into the processed folder.

In [ ]:
customer_transformed.write.format("parquet").mode("overwrite").save(paths["customer_processed"])
